In [8]:
from ngsolve import *
from netgen.occ import *
from ngsolve import Draw
from netgen.gui import *

import math

# Parameters
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilheight = nwindings * (2 * wireradius) * 1.2  # Length of the coil
coilradius = 0.01  # Radius of the coil
airh = 10 * coilradius  # Dimension of the surrounding air

# Create the cylinder face for the helix path
cyl = Cylinder((0, 0, -coilheight / 2), Z, r=coilradius, h=coilheight).faces[0]

# Create the helical edge
heli = Edge(Segment((0, 0), (nwindings * 2 * math.pi, coilheight)), cyl)

# Get start and end points and tangents (optional, can be used for further modifications)
ps = heli.start
vs = heli.start_tangent
pe = heli.end
ve = heli.end_tangent

# Create the helical wire and circular face for the coil cross-section
spiral = Wire([heli])
circ = Face(Wire([Circle(ps, Y, wireradius)]))

# Generate the coil by sweeping the circular face along the helical wire
coil = Pipe(spiral, circ)

# Set mesh size and names for the coil faces
coil.faces.maxh = 0.2
coil.faces.name = "coilbnd"
coil.faces.Max(Z).name = "in"
coil.faces.Min(Z).name = "out"
coil.mat("coil")

# Calculate cross-sectional area (optional)
crosssection = coil.faces.Max(Z).mass

# Create the surrounding air box and subtract the coil
box = Box((-airh / 2, -airh / 2, -airh / 2), (airh / 2, airh / 2, airh / 2))
box.faces.name = "outer"
air = box - coil
air.mat("air")

# Combine the coil and air into a single geometry
geo = OCCGeometry(Glue([coil, air]))

# Display the geometry
Draw(geo)

specifying material 

In [21]:
coil_sigma = 5.96e7  # Leitfähigkeit der Spule (z. B. Kupfer)
air_sigma = 0#1e-15  # Sehr geringe Leitfähigkeit für Luft
print("done")

done


creating mesh

In [22]:
mesh = Mesh(
    geo.GenerateMesh(maxh= 1.5* wireradius )#/ 2)
)  # Verfeinerung für höhere Genauigkeit
mesh.Curve(1)  

 Face 1 / 9 (parameter space projection)
 Face 2 / 9 (parameter space projection)
 Face 3 / 9 (parameter space projection)
 Face 4 / 9 (parameter space projection)
 Face 5 / 9 (parameter space projection)
 Face 6 / 9 (parameter space projection)
 Face 7 / 9 (parameter space projection)
 Face 8 / 9 (parameter space projection)
 Face 9 / 9 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Curve elements, order = 1


In [12]:
mesh.GetBoundaries()

('out', 'coilbnd', 'in', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')

In [13]:
print("Measure of 'in' boundary:", Integrate(1, mesh, definedon=mesh.Boundaries("in")))
print(
    "Measure of 'out' boundary:", Integrate(1, mesh, definedon=mesh.Boundaries("out"))
)

Measure of 'in' boundary: 3.02070061020265e-06
Measure of 'out' boundary: 3.02070060971233e-06


In [ ]:
from netgen.meshing import MeshingParameters

# Save the mesh to a Netgen format (.vol or .neutral)
import netgen.meshing as ngm

ngmesh = mesh.ngmesh
ngmesh.Save("saved_mesh_small.vol")

In [2]:
from ngsolve import Mesh
from netgen.csg import *
from netgen.meshing import Mesh as NGMesh

# Load the mesh from the file
ngmesh = NGMesh()
ngmesh.Load("saved_mesh.vol")

# Convert to NGSolve mesh
mesh = Mesh(ngmesh)

In [23]:
fes = H1(mesh, order=1, definedon=mesh.Materials('coil'),dirichlet=mesh.Boundaries('out'))  # Ansatzraum für das Potential
phi = fes.TrialFunction()  # Trial-Funktion
v = fes.TestFunction()  # Test-Funktion

sigma = CoefficientFunction(
    [coil_sigma if mat == "coil" else air_sigma for mat in mesh.GetMaterials()]
)

a = BilinearForm(fes)
a += SymbolicBFI(sigma * grad(phi) * grad(v))

# Rechte Seite (Randbedingungen)
f = LinearForm(fes)
f += SymbolicLFI(500 / crosssection * v, definedon=mesh.Boundaries("in"))

In [24]:
gfu = GridFunction(fes)

# Dirichlet-Bedingung auf der äußeren Grenze
gfu.Set(0, definedon=mesh.Boundaries("outer"))

a.Assemble()
f.Assemble()

gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu, mesh, "Potential")

In [25]:
j_c = -sigma * grad(gfu)
Draw(j_c, mesh, "Stromdichte")

In [26]:
potential_in = 1/crosssection * Integrate(gfu, mesh, definedon=mesh.Boundaries("in"))
potential_out = 1/crosssection* Integrate(gfu, mesh, definedon=mesh.Boundaries("out"))
spannungsabfall = potential_in - potential_out

print("Spannungsabfall:", spannungsabfall)

Spannungsabfall: 0.9591473896159529
